# Bromocriptine FMRIPREP Data 

In [18]:
import os, glob
import pandas as pd 
import subprocess as sp

from datetime import date

In [7]:
# Variable load and setup
date = date.today()
study_folder_path="/projects/niblab/experiments/bromocriptine"
data_folder_path= os.path.join(study_folder_path, "data")
report_folder_path=os.path.join(data_folder_path, "reports")
fmriprep_path='/projects/niblab/experiments/bromocriptine/data/fmriprep'
fmriprep_folders=glob.glob(os.path.join(fmriprep_path, "sub-*/ses-*"))
fmriprep_ses1=[x.split("/")[-2] for x in fmriprep_folders if 'ses-1' in x]
fmriprep_ses2=[x.split("/")[-2] for x in fmriprep_folders if 'ses-2' in x]

---

In [23]:
def submit_fmriprep_batch(job_file, x, submit_job=False):
    #print('[INFO] batch file: %s'%job_file)
    batch_cmd='sbatch --array={}-{}%1 {}'.format(x,x,job_file)
    #print('[INFO] batch command: {}'.format(batch_cmd))
    
    # submit batch job
    if submit_job==True: 
        sp.run(batch_cmd, shell=True)        
        #print('[INFO] submitted job.')

In [24]:
# Set Parameters 
fmriprep_job_file='/projects/niblab/experiments/bromocriptine/code/bro_fmriprep.job'

for sub_id in ['sub-006', 'sub-008', 'sub-015', 'sub-018', 'sub-031',
               'sub-042', 'sub-048', 'sub-049', 'sub-054', 'sub-055', 'sub-056']:
    
    num_id=sub_id.split("-")[1].lstrip('0')

    submit_job=False # set to True when you want to run the file 
    submit_fmriprep_batch(fmriprep_job_file,num_id, submit_job)

---

## Quality Report

In [30]:
"""
# Methods
"""

def quality_report():
    
    s1_bids_subject_list=[x.split("/")[-2] for x in 
                   glob.glob(os.path.join(study_folder_path, "data/bids/sub-*/ses-1"))]

    s1_bids_subject_list=[x for x in s1_bids_subject_list ]#if x not in remove_subs]

    s2_bids_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(study_folder_path, "data/bids/sub-*/ses-2"))]

    s2_bids_subject_list=[x for x in s2_bids_subject_list]  #if x not in remove_subs]

    s1_fprep_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(study_folder_path, "data/fmriprep/sub-*/ses-1"))]


    s2_fprep_subject_list=[x.split("/")[-2] for x in 
                       glob.glob(os.path.join(study_folder_path, "data/fmriprep/sub-*/ses-2"))]
    s1_fprep_missing = [x for x in s1_bids_subject_list if x not in s1_fprep_subject_list]
    s2_fprep_missing = [x for x in s2_bids_subject_list if x not in s2_fprep_subject_list]

    fprep_missing=list(set(s1_fprep_missing + s2_fprep_missing))
    fprep_missing.sort()

    print("[INFO] %s session 1 fmriprep subjects found."%len(s1_fprep_subject_list))
    print("[INFO] %s session 1 bids subjects are not processed through fmriprep."%len(s1_fprep_missing))
    print("[INFO] subjects: %s \n\n"%s1_fprep_missing)
    print("[INFO] %s session 2 fmriprep subjects found."%len(s2_fprep_subject_list))
    print("[INFO] %s session 2 bids subjects are not processed through fmriprep."%len(s2_fprep_missing))
    print("[INFO] subjects: %s "%s2_fprep_missing)
    print("\n[INFO] set of missing subjects: %s"%fprep_missing)
    
    
    
    
"""
# Build FMRIPREP Report 

"""


def build_fmriprep_report(write_files=False):
    

    
    sessions=['ses-1', 'ses-2']
    dataframes=[]
        
    # loop through sessions
    for session in sessions:
        #print("\n[INFO] %s"%session)
        

        data_dict={} #initialize data dictionary for session

        # loop through subject set by subject
        for i in range(1,57):
            train_ct=0
            rest_ct=0
            pe_ct=0
            subject="sub-%s"%f'{i:03}'
            bids_folder=os.path.join(fmriprep_path, subject,
                                    session)

            if subject not in data_dict:
                data_dict[subject] = {}



            # get anat file and save plot
            #_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.g
            anat_img =os.path.join(fmriprep_path, subject, "anat",
                   '%s_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.g'%(subject))

            if os.path.exists(anat_img): 
                data_dict[subject]["anat"]="good"

            # get functional files and check their volume and plot images
            func_files=glob.glob(os.path.join(
                    fmriprep_path, subject, session,
                    "func/*task-*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" ))

            for func_file in func_files:
                task=func_file.split("/")[-1].split("_")[2]
                vol = sp.check_output(["fslnvols", func_file])
                vol=str(vol,'utf-8').strip("\n")


                if "resting" in task:
                    rest_ct+=1
                    var_name="%s_vol"%task
                    data_dict[subject][var_name]=vol
                elif "pe" in task:
                    pe_ct+=1
                    run=func_file.split("/")[-1].split("_")[3]
                    var_name="%s_%s_vol"%(task,run)
                    data_dict[subject][var_name]=vol
                elif "training" in task:
                    train_ct+=1
                    run=func_file.split("/")[-1].split("_")[3]
                    var_name="%s_%s_vol"%(task,run)
                    data_dict[subject][var_name]=vol

            data_dict[subject]["pe_file_ct"]=pe_ct
            data_dict[subject]["train_file_ct"]=train_ct
            data_dict[subject]["rest_file_ct"]=rest_ct

        dataframe=pd.DataFrame(data_dict).T        
        dataframes.append(dataframe)

    return dataframes;


In [31]:
quality_report()

[INFO] 41 session 1 fmriprep subjects found.
[INFO] 9 session 1 bids subjects are not processed through fmriprep.
[INFO] subjects: ['sub-006', 'sub-015', 'sub-018', 'sub-031', 'sub-048', 'sub-049', 'sub-054', 'sub-055', 'sub-056'] 


[INFO] 40 session 2 fmriprep subjects found.
[INFO] 11 session 2 bids subjects are not processed through fmriprep.
[INFO] subjects: ['sub-006', 'sub-008', 'sub-015', 'sub-018', 'sub-031', 'sub-042', 'sub-048', 'sub-049', 'sub-054', 'sub-055', 'sub-056'] 

[INFO] set of missing subjects: ['sub-006', 'sub-008', 'sub-015', 'sub-018', 'sub-031', 'sub-042', 'sub-048', 'sub-049', 'sub-054', 'sub-055', 'sub-056']


In [32]:
fmriprep_dict={}


In [33]:
dataframes=build_fmriprep_report(write_files=False)

In [34]:
s2_df=dataframes[1]
s1_df=dataframes[0]


In [35]:
s1_df.style.highlight_null('red')


,pe_file_ct,train_file_ct,rest_file_ct,task-training_run-2_vol,task-training_run-1_vol,task-resting_vol,task-pe_run-2_vol,task-pe_run-1_vol
sub-001,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan
sub-002,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan
sub-003,2,2,1,243,243,147,193,193
sub-004,2,2,1,243,243,147,193,193
sub-005,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan
sub-006,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan
sub-007,2,2,1,243,243,147,193,193
sub-008,2,2,1,243,243,147,193,193
sub-009,2,2,1,243,243,147,193,193
sub-010,2,2,1,243,243,147,193,193


In [36]:
s2_df.style.highlight_null('red')


,task-training_run-2_vol,task-training_run-1_vol,task-resting_vol,task-pe_run-2_vol,task-pe_run-1_vol,pe_file_ct,train_file_ct,rest_file_ct
sub-001,243,243,147,193,193,2,2,1
sub-002,243,243,147,193,193,2,2,1
sub-003,243,243,nan,193,193,2,2,0
sub-004,243,243,147,193,193,2,2,1
sub-005,nan,nan,nan,nan,nan,0.000000,0.000000,0.000000
sub-006,nan,nan,nan,nan,nan,0.000000,0.000000,0.000000
sub-007,243,243,147,193,193,2,2,1
sub-008,nan,nan,nan,nan,nan,0.000000,0.000000,0.000000
sub-009,243,243,147,193,193,2,2,1
sub-010,243,243,147,193,193,2,2,1


---